In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline
from langdetect import detect
import  fuzzywuzzy.fuzz as fuzz
import time
from langdetect import detect

In [3]:
import pandas as pd

In [4]:
df=pd.read_csv("IndianFoodDatasetCSV.csv")
pd.set_option('display.max_rows', 500)
print(df)

       Srno                                         RecipeName  \
0         1                               Masala Karela Recipe   
1         2  टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...   
2         3  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3         4  Gongura Chicken Curry Recipe - Andhra Style Go...   
4         5  आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...   
...     ...                                                ...   
6866  14073  गोअन मशरुम जकुटी रेसिपी - Goan Mushroom Xacuti...   
6867  14107  शकरकंदी और मेथी का पराठा रेसिपी - Sweet Potato...   
6868  14165      Ullikadala Pulusu Recipe | Spring Onion Curry   
6869  14167  Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...   
6870  14211              नवरंग दाल रेसिपी - Navrang Dal Recipe   

                                   TranslatedRecipeName  \
0                                  Masala Karela Recipe   
1                            Spicy Tomato Rice (Recipe)   
2     Ragi Semiya Upma Recipe 

In [6]:
df.drop(["RecipeName","Ingredients","Instructions"],axis=1,inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

In [7]:
start=time.time()
df["lang"]=df["TranslatedIngredients"].apply(lambda x:detect(x))
end = time.time()
print(end - start)      

33.0801568031311


In [8]:
df=df[df["lang"]=="en"]

In [9]:
df["lang"].value_counts()

lang
en    6276
Name: count, dtype: int64

In [10]:
df.to_csv('df-lan.csv',index=False)

In [11]:
df=pd.read_csv("df-lan.csv")

In [12]:
def preprocess(text):
    text=re.sub(r'[0-9]+', '', text)
    delword="Teaspoon,teaspoon,teaspoons,-,/,to taste,as needed,shredded,cup,cups,washed,sliced,finely,few,chopped,tablespoons,\
    Fresh,optional,peeled ,Cubed,for garnishing,garnishing,grams,washed, finely chopped,chopped,frying,tablespoon,deseeded,thinly,\
    ,smearing,as per taste,as required, vegans can eliminate this ingredient, roasted,coarsely pounded,\
    ,required,plain,diced,minced,mixed with warm milk,boiled,steamed,as per taste,grated,made into paste,,cooked,\
    taste,(remove seeds fibre any),(Whole),peel cut into rings,(fine variety),cleaned,but recommended,(leftover water from washing rice)\
    to cook,quartered"
    
    stopwords = {'and','as','if','it','at','is','or',"gm","for","ml","whole"}
    
    for i in delword.split(","):
        text = text.replace(i, '')
    text = re.sub(r'\s+', ' ', text)
    text= re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', text)
    text=' '.join(filter(lambda x: x.lower() not in stopwords,  text.split()))
    return text

In [13]:
df1=df.copy()

In [14]:
df1["P-Ingredients"]=df1["TranslatedIngredients"].apply(preprocess)

In [15]:
import random
def rd():
    i=random.randint(0,6321)
    x=df1.loc[i,"P-Ingredients"]
    print(x)
    print(i)
    return x

In [16]:
tester=rd()

Hara Chana soaked overnight in water, Ginger Garlic Paste, Cumin powder (Jeera), Turmeric powder (Haldi), Garam masala powder, Green Chillies , Mango (Raw) , Spinach , Mint Leaves (Pudina), Coriander (Dhania) Leaves, Gram flour (besan),Salt
5331


In [17]:
df1["flag"]=0

In [18]:
user_ingredients='Gram flour, Curd , Green Chillies made into paste, Lemon juice, Enos fruit salt,Salt , Sunflower Oil, Mustard seeds, Cumin seeds (Jeera), Asafoetida (hing), Green Chillies, Red Chilli powder, Turmeric powder, Jaggery,Salt  , Sunflower Oil, Mustard seeds, Cumin seeds, Asafoetida , Green Chillies, Red Chilli powder, Turmeric, Jaggery , Tamarind ,Salt , Coriander '
len(user_ingredients.split(","))

26

In [19]:
df1.reset_index(drop=True,inplace=True)

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6276 entries, 0 to 6275
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Srno                    6276 non-null   int64 
 1   TranslatedRecipeName    6276 non-null   object
 2   TranslatedIngredients   6276 non-null   object
 3   PrepTimeInMins          6276 non-null   int64 
 4   CookTimeInMins          6276 non-null   int64 
 5   TotalTimeInMins         6276 non-null   int64 
 6   Servings                6276 non-null   int64 
 7   Cuisine                 6276 non-null   object
 8   Course                  6276 non-null   object
 9   Diet                    6276 non-null   object
 10  TranslatedInstructions  6276 non-null   object
 11  URL                     6276 non-null   object
 12  lang                    6276 non-null   object
 13  P-Ingredients           6276 non-null   object
 14  flag                    6276 non-null   int64 
dtypes: i

In [21]:
rows=6269

In [22]:
def Cook2(Pingredients):    
        "Comparing the Recipe Ingredients and User Ingredients and decreasing count value for each Recipe ingredient missing.\
        Count describes the no. of ingredients missing from user ingredients to make that recipe."
        flag,count=0,0
        for i in set(Pingredients.split(",")):
            for j in  user_ingredients.split(","):
                if fuzz.token_set_ratio(i,j)>80:                    
                    flag=1 
                    break
            if (flag==0):
                count-=1
            else:
                flag=0
        return count

In [23]:
df1.reset_index(drop=True,inplace=True)

In [24]:
df1.drop([5743,5173,4846,4125,3766,422,80,442,583,4994,5466,5463,5440,5383,5374,4994,1571,3311,3222,2305,1571,605,423],inplace=True)

In [25]:
df.reset_index(drop=True,inplace=True)

In [26]:
df1.to_csv('df-en-processed.csv',index=False)

To Preprocess Instructions

In [27]:
df=pd.read_csv("df-en-processed.csv")

In [28]:
df.head()

,Srno,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL,lang,P-Ingredients,flag
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,en,"Karela (Bitter Gourd Pavakkai) ,Salt , Onion ,...",0
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",http://www.archanaskitchen.com/spicy-tomato-ri...,en,"rice , tomatoes, BC Belle Bhat powder, salt , ...",0
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",http://www.archanaskitchen.com/ragi-vermicelli...,en,"Rice Vermicelli Noodles (Thin), Onion , Carrot...",0
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,http://www.archanaskitchen.com/gongura-chicken...,en,"Chicken, Onion , Tomato , Green Chillies slit,...",0
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,en,"chana dal, white urad dal, red chillies, coria...",0


In [36]:
im=pd.read_csv("images.csv")

In [37]:
im.head()

,image,url
0,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/masala-karela-...
1,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/spicy-tomato-r...
2,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/ragi-vermicell...
3,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/gongura-chicke...
4,https://www.archanaskitchen.com/images/archana...,https://www.archanaskitchen.com/andhra-style-a...


In [48]:
df["image-url"]=im["image"]

In [49]:
df["URL"]=im["url"]

In [68]:
df.shape

(6255, 17)

In [69]:
im.shape

(5951, 2)

In [50]:
df.head()

,Srno,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL,lang,P-Ingredients,flag,ParsedIngredients,image-url
0,1,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,en,"Karela (Bitter Gourd Pavakkai) ,Salt , Onion ,...",0,"[{'Ingredient': 'Bitter Gourd Pavakkai', 'Quan...",https://www.archanaskitchen.com/images/archana...
1,2,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,en,"rice , tomatoes, BC Belle Bhat powder, salt , ...",0,"[{'Ingredient': '2 cups rice cooked', 'Quanti...",https://www.archanaskitchen.com/images/archana...
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,en,"Rice Vermicelli Noodles (Thin), Onion , Carrot...",0,[{'Ingredient': 'Rice Vermicelli Noodles Thin'...,https://www.archanaskitchen.com/images/archana...
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,en,"Chicken, Onion , Tomato , Green Chillies slit,...",0,"[{'Ingredient': 'Chicken', 'Quantity': '500', ...",https://www.archanaskitchen.com/images/archana...
4,5,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,en,"chana dal, white urad dal, red chillies, coria...",0,"[{'Ingredient': 'chana dal', 'Quantity': '1', ...",https://www.archanaskitchen.com/images/archana...


In [64]:
df["image-url"].value_counts()

image-url
https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Monika_Manchanda/Nandan_Kozhi__Kerala_Style_Chicken_Curry.jpg                                                4
https://www.archanaskitchen.com/images/archanaskitchen/0-Archanas-Kitchen-Recipes/2016/sept-21/Peanut_Coconut_Chutney_Recipe_No_onion_No_Garlic_Fasting_Vrat_Recipe-3.jpg    3
https://www.archanaskitchen.com/images/archanaskitchen/0-Archanas-Kitchen-Recipes/2019/Veg_Hakka_Noodles_recipe_10_1600.jpg                                                  3
https://www.archanaskitchen.com/images/archanaskitchen/0-Affiliate-Articles/sweet_ime_soda.jpg                                                                               3
https://www.archanaskitchen.com/images/archanaskitchen/Indian_Dal_Khadi_Curry/Dal_Palak_Recipe_Healthy_Spinach_Lentil_Curry-2.jpg                                            3
                                                                                                                   

In [65]:
df[df["image-url"]== "https://www.archanaskitchen.com/images/archanaskitchen/1-Author/Monika_Manchanda/Nandan_Kozhi__Kerala_Style_Chicken_Curry.jpg"]

,Srno,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL,lang,P-Ingredients,flag,ParsedIngredients,image-url
398,431,Lauki Badi Ki Sabzi (Recipe In Hindi),"1/2 gourd - peel and chop, 1/2 cup wood, oil -...",10,20,30,3,Rajasthani,Main Course,Vegetarian,"To make gourd elder greens, first roast the el...",https://www.archanaskitchen.com/kerala-chicken...,en,"gourd peel chop, wood, oil curry leaves, cumin...",0,"[{'Ingredient': 'peel and chop', 'Quantity': '...",https://www.archanaskitchen.com/images/archana...
2228,2801,Tawa Tandoori Roti Recipe,"2 cups Whole Wheat Flour,1/4 cup Curd (Dahi / ...",30,30,60,4,North Indian Recipes,Lunch,Vegetarian,"To begin making the Tawa Tandoori Roti Recipe,...",https://www.archanaskitchen.com/kerala-chicken...,en,"Wheat Flour, Curd (Dahi Yogurt),Salt ,Sunflowe...",0,"[{'Ingredient': 'Whole Wheat Flour', 'Quantity...",https://www.archanaskitchen.com/images/archana...
3353,4643,Paneer Malai Makhani Biryani Recipe,"1 cup Basmati rice,4 teaspoon Ghee,2 teaspoon ...",30,40,70,4,Indian,Dinner,High Protein Vegetarian,To begin making the Paneer Malai Makhani Birya...,https://www.archanaskitchen.com/nandan-kozhi-r...,en,"Basmati rice, Ghee, Ginger Garlic Paste, Onion...",0,"[{'Ingredient': 'Basmati rice', 'Quantity': '1...",https://www.archanaskitchen.com/images/archana...
3356,4648,Andhra Style Sorakaya Kura Recipe - Bottle Gou...,2 cups Bottle gourd (lauki) - cut into small p...,10,25,35,4,Andhra,Lunch,Vegetarian,To begin making the Sorakaya Kura Recipe we wi...,https://www.archanaskitchen.com/kerala-chicken...,en,"Bottle gourd (lauki) cut into small pieces, Ar...",0,[{'Ingredient': 'Bottle gourd lauki cut into ...,https://www.archanaskitchen.com/images/archana...


In [51]:
import re

def parse_ingredients(ingredient_str):
    parsed_ingredients = []
    ingredients = ingredient_str.split(",")  # Split by commas to separate ingredients

    # Predefined descriptors to filter out
    descriptors = ["to taste", "thinly sliced", "finely chopped", "as required", "deseeded", "-"]

    for item in ingredients:
        item = item.strip()

        # Skip unwanted items like just "-" or empty items
        if item == "-" or not item:
            continue

        # Remove any descriptors or unnecessary parts like " - to taste", " - thinly sliced", etc.
        for descriptor in descriptors:
            item = re.sub(rf"\b{descriptor}\b", "", item).strip()

        # Regex to match quantity, unit, and ingredient
        match = re.match(r"(\d+\.?\d*/?\d*)?\s*([a-zA-Z]+)?\s*(.+)", item)
        if match:
            quantity = match.group(1) or "as required"
            unit = match.group(2) or ""
            ingredient = match.group(3).strip()

            # Clean up ingredient (remove extra spaces or undesired characters)
            ingredient = re.sub(r"[^\w\s]", "", ingredient).strip()

            # Ensure the ingredient name is clean and not empty
            if ingredient and ingredient not in descriptors:
                parsed_ingredients.append({
                    "Ingredient": ingredient,
                    "Quantity": quantity.strip(),
                    "Unit": unit.strip()
                })

    return parsed_ingredients


# Apply the improved parsing function to the dataset
df["ParsedIngredients"] = df["TranslatedIngredients"].apply(parse_ingredients)


In [52]:
# # Adjusting quantities as before
# def adjust_quantities_v2(parsed_ingredients, default_servings, desired_servings):
#     adjusted_ingredients = []
#     for item in parsed_ingredients:
#         try:
#             # Adjust quantities proportionally
#             if item["Quantity"].replace(".", "", 1).isdigit():  # Check if it's a valid number
#                 adjusted_quantity = float(item["Quantity"]) * desired_servings / default_servings
#                 adjusted_ingredients.append({
#                     "Ingredient": item["Ingredient"],
#                     "Quantity": round(adjusted_quantity, 2),  # Round to 2 decimal places
#                     "Unit": item["Unit"]
#                 })
#             else:
#                 # Handle "as required" or other non-numeric quantities
#                 adjusted_ingredients.append({
#                     "Ingredient": item["Ingredient"],
#                     "Quantity": item["Quantity"],
#                     "Unit": item["Unit"]
#                 })
#         except Exception as e:
#             print(f"Error adjusting ingredient: {item} - {e}")
#     return adjusted_ingredients

In [53]:
# Example usage
recipe_index = 0  # Change to test with different recipes
default_servings = df.loc[recipe_index, "Servings"]
desired_servings = 4  # Example input
parsed_ingredients = df.loc[recipe_index, "ParsedIngredients"]
original_ingredients = df.loc[recipe_index, "TranslatedIngredients"]
# Display results
df.iloc[recipe_index]
print("Original Ingredients:", original_ingredients)
print("Parsed Ingredients:", parsed_ingredients)

Original Ingredients: 6 Karela (Bitter Gourd/ Pavakkai) - deseeded,Salt - to taste,1 Onion - thinly sliced,3 tablespoon Gram flour (besan),2 teaspoons Turmeric powder (Haldi),1 tablespoon Red Chilli powder,2 teaspoons Cumin seeds (Jeera),1 tablespoon Coriander Powder (Dhania),1 tablespoon Amchur (Dry Mango Powder),Sunflower Oil - as required
Parsed Ingredients: [{'Ingredient': 'Bitter Gourd Pavakkai', 'Quantity': '6', 'Unit': 'Karela'}, {'Ingredient': 'Gram flour besan', 'Quantity': '3', 'Unit': 'tablespoon'}, {'Ingredient': 'Turmeric powder Haldi', 'Quantity': '2', 'Unit': 'teaspoons'}, {'Ingredient': 'Red Chilli powder', 'Quantity': '1', 'Unit': 'tablespoon'}, {'Ingredient': 'Cumin seeds Jeera', 'Quantity': '2', 'Unit': 'teaspoons'}, {'Ingredient': 'Coriander Powder Dhania', 'Quantity': '1', 'Unit': 'tablespoon'}, {'Ingredient': 'Amchur Dry Mango Powder', 'Quantity': '1', 'Unit': 'tablespoon'}, {'Ingredient': 'Oil', 'Quantity': 'as required', 'Unit': 'Sunflower'}]


In [70]:
import json
# Use a working placeholder image URL
PLACEHOLDER_IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/a/ac/No_image_available.svg"
# Prepare recipe data
def prepare_recipe_data(df):
    recipe_data = {}
    for _, row in df.iterrows():
        recipe_name = row["TranslatedRecipeName"]
        default_servings = row["Servings"]
        ingredients = parse_ingredients(row["TranslatedIngredients"])

        course= row["Course"]
        diet=row["Diet"]
        # Handle NaN values and replace with defaults
        image_url = row["image-url"] if pd.notna(row["image-url"]) else PLACEHOLDER_IMAGE_URL

        recipe_data[recipe_name] = {
            "DefaultServings": default_servings,
            "image-url": image_url,
            "course" : course,
            "diet": diet,
            "Ingredients": ingredients
            
        }
    return recipe_data

# Save recipe data to JSON file
def save_to_json(data, filename):
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

In [71]:
# Process the dataset and save it as JSON
recipe_data = prepare_recipe_data(df)
save_to_json(recipe_data, "recipe_data.json")

print("Recipe data has been saved to 'recipe_data.json'")

Recipe data has been saved to 'recipe_data.json'


In [ ]:
do more less you wish,cut into squares,cut into pieces,cut into cubes,cut into small cubes,per use, coarsely crushed, slit lengthwise,wash well in water, cut into half, make puree,to smear, cut to small florets, soaked in hot water,roughly thin,roughly thin,into inch square pieces,granulated

SyntaxError: invalid syntax (<ipython-input-25-97055e7bcdf9>, line 1)

,Unnamed: 0,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,P-Ingredients,flag,image-url
398,398,Kerala Chicken Curry Recipe With Freshly Groun...,"500 Chicken - medium sized pieces,1 Onion - ro...",55,Kerala Recipes,To prepare Kerala Chicken Curry With Freshly G...,https://www.archanaskitchen.com/kerala-chicken...,"chicken medium sized pieces,onion roughly,ging...",-10,https://www.archanaskitchen.com/images/archana...
2228,2228,Kerala Chicken Curry Recipe - Nadan Kozhi Curry,"500 grams Chicken - curry-cut with bones,2 tea...",40,Kerala Recipes,"To begin making Kerala Chicken Curry Recipe, w...",https://www.archanaskitchen.com/kerala-chicken...,"chicken currycut with bones,red chilli powder,...",-9,https://www.archanaskitchen.com/images/archana...
3353,3353,Nandan Kozhi Recipe - Kerala Style Chicken Curry,"500 grams Chicken - curry cut,3 Onions - slice...",50,Kerala Recipes,"To begin making the Nandan Kozhi recipe, first...",https://www.archanaskitchen.com/nandan-kozhi-r...,"chicken curry cut,onion,tomatoes,ginger,cloves...",-8,https://www.archanaskitchen.com/images/archana...
3356,3356,Kerala Chicken Curry Recipe,400 grams Chicken - cut into medium sized piec...,70,Kerala Recipes,"To prepare the Kerala Chicken Curry, in a mixi...",https://www.archanaskitchen.com/kerala-chicken...,"chicken cut into medium sized pieces,curd (dah...",-11,https://www.archanaskitchen.com/images/archana...
